<a href="https://colab.research.google.com/github/Blue-coder89/Machine-Learning/blob/main/OwnTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_model,num_heads):
    super(MultiHeadAttention,self).__init__()
    ## Ensure that the model dimension is divisible by the number of heads
    assert d_model%num_heads == 0 , "d_model must be divisible by num_heads"
    ## Initialize dimensions
    self.d_model = d_model ## Model dimension
    self.num_heads = num_heads ## number of attention heads
    self.d_k = d_model//num_heads ## Dimension of each head's key,query,value
    ## Linear model for transforming input
    self.W_q = nn.Linear(d_model,d_model) ## Query transformation
    self.W_k = nn.Linear(d_model,d_model) ## Key transformation
    self.W_v = nn.Linear(d_model,d_model) ## Value transformation
    self.W_o = nn.Linear(d_model,d_model) ## Output transformation

  def split_heads(self,x):
    ## Reshape the input for num_heads to have multi-head attention
    batch_size,seq_length,d_model = x.size()
    return x.view(batch_size,seq_length,self.num_heads,self.d_k).transpose(1,2)

  def combine_heads(self,x):
    ## combine the multiple heads back to original shape
    batch_size,_,seq_len,d_k = x.size()
    return x.transpose(1,2).contigous().view(batch_size,seq_len,self.d_model)

  def scaled_dot_product_attention(self,Q,K,V,mask=None):
    ## calculate attention scores
    attn_scores = torch.matmul(Q,K.transpose(-2,-1))/math.sqrt(self.d_k)
    ## apply mask if provided. Useful to prevent attention to certain parts like padding
    if mask is not None:
      attn_scores = attn_scores.masked_fill(mask==0,-1e9)
    ## finding probabilities using softmax
    attn_probs = torch.softmax(attn_scores,dim=-1)
    ## multiplying by values to obtain the final output
    return torch.matmul(attn_probs,V)

  def forward(self,Q,K,V,mask=None):
    ## Apply linear transformation and split heads
    Q = self.split_heads(self.W_q(Q))
    K = self.split_heads(self.W_k(K))
    V = self.split_heads(self.W_v(V))

    ## performed scaled dot product
    attn_output = self.scaled_dot_product_attention(Q,K,V,mask)

    output = self.W_o(self.combine_heads(attn_output))
    return output




In [ ]:
class PositionWiseFeedForward(nn.Module):
  def __init__(self,d_model,d_ff):
    super().__init__()
    self.fc1 = nn.Linear(d_model,d_ff)
    self.fc2 = nn.Linear(d_ff,d_model)
    self.relu = nn.ReLU()

  def forward(self,x):
    self.fc2(self.relu(self.fc1))

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self,d_model,max_seq_length):
    super().__init__()

    pe = torch.zeros(max_seq_length,d_model)
    position = torch.arange(0,max_seq_length,dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0,d_model,2).float()* -(math.log(10000.0)/d_model))
    pe[:,0::2] = torch.sin(position*div_term)
    pe[:,1::2] = torch.cos(position*div_term)

    self.register_buffer('pe',pe.unsqueeze(0))

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self,d_model,num_heads,d_ff,dropout):
    super().__init__()
    self.self_attn = MultiHeadAttention(d_model,num_heads)
    self.feed_forward = PositionWiseFeedForward(d_model,d_ff)
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x,mask):
    attn_output = self.self_attn(x,x,x,mask)
    x = self.norm1(x + self.dropout(attn_output))
    ff_output = self.feed_forward(x)
    x = self.norm2(x + self.dropout(ff_output))
    return x


In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self,d_model,num_heads,d_ff,dropout):
    super().__init__()
    self.self_attn = MultiHeadAttention(d_model,num_heads)
    self.cross_attn = MultiHeadAttention(d_model,num_heads)
    self.feed_forward = PositionWiseFeedForward(d_model,d_ff)
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.norm3 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x,enc_output,src_mask,tgt_mask):
    attn_output = self.self_attn(x,x,x,tgt_mask)
    x = self.norm1(x + self.dropout(attn_output))
    attn_output = self.cross_attn(x,enc_output,enc_output,src_mask)
    x = self.norm2(x + self.dropout(attn_output))
    ff_output = self.feed_forward(x)
    x = self.norm3(x + self.dropout(ff_output))
    return x

In [ ]:
class Transformer(nn.Module):
  def __init__(self,src_vocab_size,tgt_vocab_size,d_model,num_heads,num_layers,d_ff,max_seq_length,dropout):
    super().__init__()
    self.encoder_embedding = nn.Embedding(src_vocab_size,d_model)
    self.decoder_embedding = nn.Embedding(tgt_vocab_size,d_model)
    self.positional_encoding = PositionalEncoding(d_model,max_seq_length)

    self.encoder_layers = nn.ModuleList([EncoderLayer(d_model,num_heads,d_ff,dropout) for _ in range(num_layers)])
    self.decoder_layers = nn.ModuleList([DecoderLayer(d_model,num_heads,d_ff,dropout) for _ in range(num_layers)])

    self.fc = nn.Linear(d_model,tgt_vocab_size)
    self.dropout = nn.Dropout(dropout)

  def generate_mask(self,src,tgt):
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
    seq_length = tgt.size(1)
    nopeak_mask = (1 - torch.triu(torch.ones(1,seq_length,seq_length),diagonal=1)).bool()
    tgt_mask = tgt_mask & nopeak_mask
    return src_mask,tgt_mask

  def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output
